In [ ]:
import torch

if torch.cuda.is_available:
    device = "cuda"
else:
  device = "cpu"

print(device)

cuda


In [ ]:
import torch
print(torch.cuda.is_available())


False


# import

## down

In [1]:
from google.colab import drive
drive.mount("/content/myDrive")

Mounted at /content/myDrive


In [2]:
import pandas as pd

data1 = pd.read_csv("/content/myDrive/MyDrive/life/EXplore/мероприятия/Марафон \"Цифровой Петербург\"/FuzzySearch/данные/fias_dict.csv")
data2 = pd.read_csv("/content/myDrive/MyDrive/life/EXplore/мероприятия/Марафон \"Цифровой Петербург\"/FuzzySearch/данные/ojf_dict2.csv")

In [3]:
data = pd.concat([data1, data2], axis=0, ignore_index=True)
print(len(data1))
print(len(data2))
print(len(data))

3475306
3514963
6990269


In [ ]:
print(data.iloc[:8])
print(data.iloc[-8:])
print(len(data))

     id                                            address
0  4812  Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...
1  4813  Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...
2  4814  Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...
3  4815  Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...
4  4816  Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...
5  4817  Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...
6  4818  Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...
7  4819  Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...
           id                                            address
6990261  1952  Санкт-Петербург г, пр-кт. Литейный, д. 59, лит...
6990262  1953  Санкт-Петербург г, пр-кт. Литейный, д. 59, лит...
6990263  1954  Санкт-Петербург г, пр-кт. Литейный, д. 59, лит...
6990264  1955  Санкт-Петербург г, пр-кт. Литейный, д. 59, лит...
6990265  1956  Санкт-Петербург г, пр-кт. Литейный, д. 59, лит...
6990266  1957  Санкт-Петербург г, пр-кт. Литейный, д. 59, лит...
6990267  1958 

In [ ]:
data.head()

,id,address
0,4812,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
1,4813,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
2,4814,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
3,4815,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
4,4816,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."


In [ ]:
data.tail(10)

,id,address
6990259,1950,"Санкт-Петербург г, пр-кт. Литейный, д. 59, лит..."
6990260,1951,"Санкт-Петербург г, пр-кт. Литейный, д. 59, лит..."
6990261,1952,"Санкт-Петербург г, пр-кт. Литейный, д. 59, лит..."
6990262,1953,"Санкт-Петербург г, пр-кт. Литейный, д. 59, лит..."
6990263,1954,"Санкт-Петербург г, пр-кт. Литейный, д. 59, лит..."
6990264,1955,"Санкт-Петербург г, пр-кт. Литейный, д. 59, лит..."
6990265,1956,"Санкт-Петербург г, пр-кт. Литейный, д. 59, лит..."
6990266,1957,"Санкт-Петербург г, пр-кт. Литейный, д. 59, лит..."
6990267,1958,"Санкт-Петербург г, пр-кт. Литейный, д. 59, лит..."
6990268,1959,"Санкт-Петербург г, пр-кт. Литейный, д. 59, лит..."


## pre dataset

## label2id

In [4]:
label2id = {
    "O": 0,
    "B-улица": 1,
    "I-улица": 2,
    "B-дом": 3,
    "I-дом": 4,
    "B-корпус": 5,
    "I-корпус": 6,
    "B-литера": 7,
    "I-литера": 8,
    "B-квартира": 9,
    "I-квартира": 10,
}
id2label = {v: k for k, v in label2id.items()}

In [5]:
import re

def get_label1(text):
    text = str(text).lower().strip()

    if 'санкт' in text:
        return label2id["O"]

    elif re.search(r',', text) or "." in text:
        return label2id["O"]

    elif re.search(r'(пр|проспект|пер|ул\.?|улица|б-р|бул\.?|пл\.?)', text):
        return label2id["B-улица"]

    elif re.fullmatch(r'\d+', text) or re.fullmatch(r'\d+-\d+', text) or 'д.' in text:
        return label2id["B-дом"]

    elif re.search(r'кв|квартира|помещ', text):
        return label2id["B-квартира"]

    elif re.search(r'литер', text):
        return label2id["B-литера"]

    elif re.search(r'к?', text):
        return label2id["B-корпус"]



    else:
        return label2id["O"]  # неизвестное или пустое




## ds creation

In [6]:
data_shf = data.sample(frac=1, random_state=43).reset_index(drop=True)

In [7]:
data_shf.head(20)

,id,address
0,1509887,"Санкт-Петербург г, пр-кт Большевиков, 9 к. 3 л..."
1,2632735,"Санкт-Петербург г, ул Маршала Казакова, 32 лит..."
2,2598050,"Санкт-Петербург г, ул Манчестерская, 3 стр. 1 ..."
3,1103111,"Санкт-Петербург г, п. Парголово, ул. Пионерска..."
4,1885927,"Санкт-Петербург г, проезд. 1-й Предпортовый, д..."
5,1762216,"Санкт-Петербург г, ул. Софийская, 4 литера Р ,..."
6,427646,"Санкт-Петербург г, ул Политрука Пасечника, 1 к..."
7,576019,"Санкт-Петербург г, ул Адмирала Черокова, 18 к...."
8,1333411,"Санкт-Петербург г, ул Хошимина, 9 к. 2 литера ..."
9,1104641,"Санкт-Петербург г, п. Парголово, пр-д. Толубее..."


In [ ]:
#@title 1
import pandas as pd

data_words = []
data_tags = []

for i in range(len(data[:200000])):
    value = str(data.iloc[i, 1])

    if "к." in value:
        pos = value.find("к.")
        value = value[:pos-1] + "," + value[pos-1:]

    if "литера" in value:
        pos = value.find("литера")
        value = value[:pos-1] + "," + value[pos-1:]

    parts = [part.strip() for part in value.split(",") if part.strip() != ""]

    words_row = []
    tags_row = []

    for part in parts:
        text = str(part).strip()

        words_field = text.split()
        if not words_field:
                continue

        if "к." in part:
            words_row.append("к")
            tags_row.append(5)
            words_row.append(".")
            tags_row.append(0)
            words_row.append(part[part.index("к.") + 2:])
            tags_row.append(6)
            words_row.extend(",")
            tags_row.append(0)
            continue

        elif "кв." in part:
            words_row.append("кв")
            tags_row.append(9)
            words_row.append(".")
            tags_row.append(0)
            words_row.append(part[part.index("кв.") + 3:])
            tags_row.append(10)
            continue

        elif "помещ." in part:
            words_row.append("помещ")
            tags_row.append(9)
            words_row.append(".")
            tags_row.append(0)
            words_row.append(part[part.index("помещ.") + 6:])
            tags_row.append(10)
            continue

        tg = get_label1(words_field[0])
        tags_field = [tg]

        for word in words_field[1:]:
            tags_field.append(tg + 1 if tg != 0 else 0)

        words_row.extend(words_field)
        tags_row.extend(tags_field)
        words_row.extend(",")
        tags_row.append(0)

    data_words.append(words_row)
    data_tags.append(tags_row)

# Финальные датафреймы
data_words_df = pd.DataFrame(data_words).fillna(0)
data_tags_df = pd.DataFrame(data_tags).fillna(0).astype(int)



In [ ]:
x = 2000

data_words_df.iloc[x:x+50]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
2000,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,21,0
2001,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,317,0
2002,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,439,0
2003,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",помещ,.,30-Н,0
2004,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,527,0
2005,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,70,0
2006,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,49,0
2007,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,310,0
2008,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,136,0
2009,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,318,0


In [ ]:
data_tags_df.iloc[x:x+50]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
2000,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2001,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2002,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2003,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2004,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2005,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2006,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2007,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2008,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2009,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0


# export

In [ ]:
data_words_df = data_words_df.drop(data_words_df.columns[20:50], axis=1)
data_tags_df = data_tags_df.drop(data_tags_df.columns[20:50], axis=1)

In [ ]:
data_words_df.to_csv("0_orig_dswords_shf.csv")
data_tags_df.to_csv("0_orig_dstags_shf.csv")